## Gait Video Study 
### Identifying frames with HSRs in each video for each cohort and trial to establish break points and also evaluate the corresponding HSR labelling via the ground truth available. Further, downsample with smoothing to define fixed shape of the input tensor for models. 
#### Remember to preserve the original count of frames in a single stride (before down sampling via smoothing) for each stride to add as an additional artificial feature later to add information about speed of the subject to the model

In [1]:
import numpy as np
import cv2
import os
import glob
%matplotlib widget
import matplotlib.pyplot as plt
import pandas as pd
import time
import shutil
import scipy
from scipy import signal
import warnings
warnings.filterwarnings("ignore")
from IPython.display import display, HTML

In [2]:
path = 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data'
frame_path_merged = 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\multi_view_merged_data\\'

#Configuration for which to run the code for 
cohorts = ['\\HOA', '\\MS', '\\PD', '\\ExtraHOA']
trials = ['\\beam_walking', '\\walking']
cameras = ['\\feet\\', '\\lower_body\\']

order = ['right hip', 'right knee', 'right ankle', 'left hip', 'left knee', 'left ankle', 'left toe 1', 'left toe 2', \
         'left heel', 'right toe 1', 'right toe 2', 'right heel']

### Utility functions 

In [5]:
#Saving the HSRframes.txt file to the hip_height_normalized\\ containing the final .csvs for analysis
for cohort in cohorts:
    for trial in trials:
        merged_path = frame_path_merged+cohort+trial 
        if (os.path.exists(merged_path)):
            videos = os.listdir(merged_path)
#             print (len(videos))
        for video in videos:
            print (glob.glob(path+cohort+trial+'\\feet\\'+'Inked'+video+'_0_Trim'))
            try:
                if (not os.path.exists(merged_path+'\\'+video+'\\HSRframes.txt')):
                    HSR_frames_file = path+cohort+trial+'\\feet\\'+'Inked'+video+'_0_Trim'+'\\HSRframes.txt'
                    shutil.copy(HSR_frames_file, merged_path+'\\'+video+'\\hip_height_normalized\\') 
                    print ('HSR for', video, 'copied')
                else:
                    print ('HSR for', video, 'exists')
            except Exception as e:
                print (e)

[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data\\HOA\\beam_walking\\feet\\InkedGVS_212_T_T1_0_Trim\\HSRframes.txt'
[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data\\HOA\\beam_walking\\feet\\InkedGVS_212_T_T2_0_Trim\\HSRframes.txt'
[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data\\HOA\\beam_walking\\feet\\InkedGVS_213_T_T1_0_Trim\\HSRframes.txt'
[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data\\HOA\\beam_walking\\feet\\InkedGVS_213_T_T2_0_Trim\\HSRframes.txt'
[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Project\\GaitVideoData\\video\\frame_data\\HOA\\beam_walking\\feet\\InkedGVS_214_T_T1_0_Trim\\HSRframes.txt'
[]
[Errno 2] No such file or directory: 'C:\\Users\\purpl\\Box\\Gait Video Proje

In [15]:
#Function to plot the given feature across frames of a video and the corresponding ground truth HSRs
def plot_true_HSR(signal, index, HSRindices, video, name):
    fig= plt.figure(figsize = (8.5, 3))
    ax1 = fig.add_subplot(111)
    ax1.plot(index, signal, color = 'b', ls='solid', marker='*', \
            markerfacecolor = 'r', markeredgecolor = 'r', ms = 10, markevery= HSRindices, label = 'true HSR')
    plt.xticks(index[0::40], fontsize = 8)
    plt.legend()
    plt.title(name)
    plt.show()
    if not os.path.exists('HSR_detection_figs\\'+ video):
        os.mkdir('HSR_detection_figs\\'+ video)
    plt.savefig('HSR_detection_figs\\'+ video+ '\\' + video+ '_'+ name + '.png', dpi = 250)
#     append = np.append(ax.get_xticks(), trueHSR_list)
#     # ax.set_xticks(trueHSR_list)
#     # ax.set_xticks(temp.index)
#     plt.show()

In [23]:
def plots(identification_feature, series, series_complete, trueHSR_indices, trueHSR_indices_complete, video):
    print (series.isna().sum(), series_complete.isna().sum())
    #Plotting the right heel height (not containing placeholders for the missing frames) and ground truth HSR 
    plot_true_HSR(series.values, series.index, trueHSR_indices, video, identification_feature)

    #Plotting the filterd signal (not containing placeholders for the missing frames) and ground truth HSR 
    filtered_values = signal.savgol_filter(series.values, window_length = 5, polyorder=3, deriv = 1)
    plot_true_HSR(filtered_values, series.index, trueHSR_indices, video, 'filtered '+identification_feature)

#     #Plotting the right heel height (containing placeholders for the missing frames) and ground truth HSR 
#     plot_true_HSR(series_complete.values, series_complete.index, trueHSR_indices_complete, video, \
#                   'complete '+identification_feature)

#     #Plotting the filterd signal (containing placeholders for the missing frames) and ground truth HSR 
#     filtered_values_complete = signal.savgol_filter(series_complete.values, \
#                                                     window_length = 11, polyorder=3)
#     plot_true_HSR(filtered_values_complete, series_complete.index, trueHSR_indices_complete, video, \
#                   'filtered complete '+identification_feature)

In [24]:
def feature_plots(video_features, video_features_complete):
      #Right Heel Height 
    series = video_features['right heel-z']
    series_complete = video_features_complete['right heel-z']
    plots('right heel-z', series, series_complete, trueHSR_indices, trueHSR_indices_complete, video)

    #Right Toe 1 z - Right heel height z
    series = video_features['right toe 1-z']-video_features['right heel-z']
    series_complete = video_features_complete['right toe 1-z'] - video_features_complete['right heel-z']
#     plots('right toe 1-z - right heel-z', series, series_complete, trueHSR_indices, trueHSR_indices_complete, video)

    #Right Toe 2 z - Right heel height z
    series = video_features['right toe 2-z']-video_features['right heel-z']
    series_complete = video_features_complete['right toe 2-z'] - video_features_complete['right heel-z']
#     plots('right toe 2-z - right heel-z', series, series_complete, trueHSR_indices, trueHSR_indices_complete, video)

    #Right Toe 2 - Right heel y coordinate
    series = video_features['right toe 2-y']-video_features['right heel-y']
    series_complete = video_features_complete['right toe 2-y'] - video_features_complete['right heel-y']
#     plots('right toe 2-y- right heel-y', series, series_complete, trueHSR_indices, trueHSR_indices_complete, video)

    #Right Toe 1 height 
    series = video_features['right toe 1-z']
    series_complete = video_features_complete['right toe 1-z']
    plots('right toe 1-z', series, series_complete, trueHSR_indices, trueHSR_indices_complete, video)

    #Right Toe 2 height 
    series = video_features['right toe 2-z']
    series_complete = video_features_complete['right toe 2-z']
#     plots('right toe 2-z', series, series_complete, trueHSR_indices, trueHSR_indices_complete, video)


In [25]:
labels = ['frame_number'] + [o + '-'+ y for o in order for y in ['x', 'y', 'z']]

for cohort in cohorts[1:2]:
    for trial in trials[:1]:
        merged_path = frame_path_merged+cohort+trial 
        if (os.path.exists(merged_path)):
            videos = os.listdir(merged_path)
#             print (len(videos))
        for video in videos[3:4]:
            #Reading the ground truth HSR frames numbers
            trueHSR = open(merged_path+'\\'+video+'\\hip_height_normalized\\'+'\\HSRframes.txt').read()
            #Making a list containing ground truth HSRs out of the read file
            trueHSR_list = [int(a) for a in trueHSR.split(',')]
            print (trueHSR_list)
            
            frames = glob.glob(merged_path+'\\'+video+'\\hip_height_normalized\\*.csv')
            #First, we need to sort the frames since we need frames to appear in order to detect HSRs
            sorted_frames = sorted(frames,  key=lambda name: int(name.split('\\')[-1][:-4]))
            #Dataframe to hold all 12*3 features (right hip-x, right hip-y, ...) as columns and all frames as rows 
            #for each video 
            video_features = pd.DataFrame(columns = labels)
            for frame in sorted_frames:
                #Append the frame number also in the temp dataframe, since it the true HSR is given in frame number
                frame_no = int(frame.split('\\')[-1][:-4])
                frame_csv = pd.read_csv(frame, index_col = 0)
                #Appending the 36 features and frame number of each frame as a row for each video's dataframe 
                video_features.loc[len(video_features)] = np.append(frame_no, frame_csv[['x', 'y', 'z']].values.flatten())
            video_features = video_features.astype({'frame_number': 'int'})
            #Setting the frame number as the index 
            video_features.set_index('frame_number', inplace = True)
            display(video_features)
            
            #If some frames are missing, appending those rows with NaN values 
            video_features_complete = video_features.reindex(range(video_features.index[-1]))
            #Indices for the ground truth HSR in the video_features dataframe (not containing placeholders for 
            #the missing frames)
            trueHSR_indices = [i for i, val in enumerate(video_features.index) if val in trueHSR_list] 
            #Indices for the ground truth HSR in the video_features_complete dataframe (containing NaN valued placeholders for 
            #the missing frames)
            trueHSR_indices_complete = [i for i, val in enumerate(video_features_complete.index) if val in trueHSR_list] 
            feature_plots(video_features, video_features_complete)
  

[53, 100, 139, 180, 217, 258, 301, 346, 387, 428, 468, 507, 547, 584, 620, 656, 690, 726, 757, 791, 822, 857, 890, 924, 956, 991, 1026, 1062, 1094, 1125, 1161, 1196, 1227, 1261, 1295, 1329, 1361, 1395, 1429, 1462, 1494, 1529, 1561, 1595, 1627, 1657, 1690, 1719, 1751]


,right hip-x,right hip-y,right hip-z,right knee-x,right knee-y,right knee-z,right ankle-x,right ankle-y,right ankle-z,left hip-x,...,left heel-z,right toe 1-x,right toe 1-y,right toe 1-z,right toe 2-x,right toe 2-y,right toe 2-z,right heel-x,right heel-y,right heel-z
frame_number,,,,,,,,,,,,,,,,,,,,,
0,41.370643,163.228161,99.959804,36.643326,145.290165,86.960413,32.404607,107.722605,24.782969,19.590266,...,28.296674,33.887073,123.654815,14.224877,38.345958,120.902105,16.063455,29.913310,114.414536,20.338801
1,41.489810,164.325712,99.729965,36.727253,145.644953,87.163416,32.487928,107.963289,24.853433,19.637059,...,28.359521,33.037907,122.701965,15.083775,38.420932,121.197506,16.097618,29.993543,114.673994,20.396808
2,40.728796,164.768122,99.952857,36.003600,147.928942,86.103504,32.623244,109.553953,24.038792,19.701044,...,27.544598,34.961160,124.268777,14.290404,39.432922,121.495981,16.142338,30.088584,115.009466,20.414202
3,40.159973,160.651475,100.195847,35.485498,143.326761,86.965627,32.216717,108.267210,23.742999,18.541376,...,18.509690,33.672419,122.835225,14.089248,39.007619,121.332571,15.101749,29.756769,114.918405,19.325933
4,40.395077,162.842572,99.730411,35.712094,144.947790,86.760146,33.211226,107.410946,24.769385,18.602010,...,20.235141,33.775890,122.052749,15.063996,39.086210,119.281363,16.917555,30.704375,112.816721,21.177855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1735,41.515564,137.451876,100.000000,35.742624,148.646568,84.653613,32.739362,149.780361,13.903912,11.567446,...,9.995548,33.273470,164.773624,3.965422,41.238126,163.517675,4.822387,28.590621,156.315481,9.558647
1736,42.097287,133.329501,100.000000,34.999575,147.899273,87.251615,34.634767,142.221940,21.025555,12.251690,...,4.321840,35.870013,156.119137,11.815865,40.901954,153.848687,13.336467,31.870934,146.571769,18.133303
1737,42.099438,133.322825,100.000000,35.005352,147.911365,87.961349,34.707677,144.347646,20.334225,12.244748,...,4.320795,35.862753,156.155656,12.509365,40.936819,154.941229,13.329981,31.911538,147.645839,18.139080


0 194


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0 194


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
#Do we need the missing frame information 
#We might need to annotate the ground truth HSR frames in the plot 
#https://stackoverflow.com/questions/14432557/matplotlib-scatter-plot-with-different-text-at-each-data-point
#For any algorithm to find the HSR's, we might need to fill missing values in the series we are using to detect HSR
#The ground truth HSR markers are not shown when the values are NAN- solve?



In [21]:
video_features.iloc[trueHSR_indices]

,right hip-x,right hip-y,right hip-z,right knee-x,right knee-y,right knee-z,right ankle-x,right ankle-y,right ankle-z,left hip-x,...,left heel-z,right toe 1-x,right toe 1-y,right toe 1-z,right toe 2-x,right toe 2-y,right toe 2-z,right heel-x,right heel-y,right heel-z
frame_number,,,,,,,,,,,,,,,,,,,,,
53,37.048133,146.451724,100.24585,36.356229,138.623370,82.602680,34.803705,123.244309,12.885191,11.237682,...,13.492850,39.657276,142.701991,0.000095,42.057920,139.010111,2.455036,32.509905,129.407018,8.792484
100,39.074558,129.882936,100.00000,33.794004,135.428224,90.817186,32.443192,142.257810,16.296373,13.566014,...,6.369884,36.258862,166.856018,0.000083,37.589039,163.603924,2.160192,30.502734,148.731697,11.998443
139,36.820152,128.769127,100.00000,32.092320,148.742838,94.864718,32.647042,168.016627,12.099557,6.272163,...,6.406413,32.593489,186.266721,0.000083,36.882348,184.024354,1.499800,31.394341,173.434898,8.503569
180,37.551827,128.745619,100.00000,34.658947,138.520406,90.134863,33.357162,147.590420,14.128858,8.518414,...,18.099223,38.514812,168.925415,0.000083,41.301464,164.610357,2.869353,30.567825,150.906839,11.921651
217,38.339703,130.495360,100.00000,32.186540,130.781715,89.025650,31.009072,142.292728,11.401710,12.136953,...,37.677862,29.434787,159.483218,0.000083,35.165033,157.227221,1.513235,31.142704,146.571348,8.565510
301,38.312743,129.309013,100.00000,34.221073,126.831025,87.932107,32.599775,126.146770,18.391939,13.747978,...,13.390751,39.442292,153.919775,0.000083,40.107495,151.719196,1.461037,28.414938,131.601595,14.762770
346,35.264289,125.281756,100.00000,32.912077,130.762056,88.247174,32.377453,140.096116,12.068455,8.422134,...,24.712305,35.219363,158.312626,0.000083,38.807329,156.038923,1.518436,31.121635,145.496496,8.484320
387,38.993032,128.124045,100.00000,34.632111,138.523948,87.974323,34.814677,147.554275,11.999158,10.640862,...,15.536359,38.355923,172.570379,6.916573,40.514652,163.545678,1.414694,32.159665,154.084736,7.661540
428,38.342386,130.492127,100.00000,35.619649,145.525640,85.552791,35.792570,153.980679,9.958998,10.749334,...,18.214282,36.326403,169.004747,0.000083,40.685229,167.812071,0.804097,33.778980,158.348917,7.056832


In [22]:
video_features

,right hip-x,right hip-y,right hip-z,right knee-x,right knee-y,right knee-z,right ankle-x,right ankle-y,right ankle-z,left hip-x,...,left heel-z,right toe 1-x,right toe 1-y,right toe 1-z,right toe 2-x,right toe 2-y,right toe 2-z,right heel-x,right heel-y,right heel-z
frame_number,,,,,,,,,,,,,,,,,,,,,
0,41.370643,163.228161,99.959804,36.643326,145.290165,86.960413,32.404607,107.722605,24.782969,19.590266,...,28.296674,33.887073,123.654815,14.224877,38.345958,120.902105,16.063455,29.913310,114.414536,20.338801
1,41.489810,164.325712,99.729965,36.727253,145.644953,87.163416,32.487928,107.963289,24.853433,19.637059,...,28.359521,33.037907,122.701965,15.083775,38.420932,121.197506,16.097618,29.993543,114.673994,20.396808
2,40.728796,164.768122,99.952857,36.003600,147.928942,86.103504,32.623244,109.553953,24.038792,19.701044,...,27.544598,34.961160,124.268777,14.290404,39.432922,121.495981,16.142338,30.088584,115.009466,20.414202
3,40.159973,160.651475,100.195847,35.485498,143.326761,86.965627,32.216717,108.267210,23.742999,18.541376,...,18.509690,33.672419,122.835225,14.089248,39.007619,121.332571,15.101749,29.756769,114.918405,19.325933
4,40.395077,162.842572,99.730411,35.712094,144.947790,86.760146,33.211226,107.410946,24.769385,18.602010,...,20.235141,33.775890,122.052749,15.063996,39.086210,119.281363,16.917555,30.704375,112.816721,21.177855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1735,41.515564,137.451876,100.000000,35.742624,148.646568,84.653613,32.739362,149.780361,13.903912,11.567446,...,9.995548,33.273470,164.773624,3.965422,41.238126,163.517675,4.822387,28.590621,156.315481,9.558647
1736,42.097287,133.329501,100.000000,34.999575,147.899273,87.251615,34.634767,142.221940,21.025555,12.251690,...,4.321840,35.870013,156.119137,11.815865,40.901954,153.848687,13.336467,31.870934,146.571769,18.133303
1737,42.099438,133.322825,100.000000,35.005352,147.911365,87.961349,34.707677,144.347646,20.334225,12.244748,...,4.320795,35.862753,156.155656,12.509365,40.936819,154.941229,13.329981,31.911538,147.645839,18.139080


### Downsample with smoothing to define fixed shape input tensor for models

In [ ]:
#Use mean with disjoint windows to downsample while smoothing 
#Make sure to preserve count of frames in a frame before smoothing to add as a feature 

